In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import sqlite3
import gravis as gv
from sklearn.cluster import DBSCAN, KMeans
import pygraphviz
# from pyvis.network import Network
# import graph_tool.all as gt
import matplotlib.pyplot as plt
from itertools import combinations
%config InlineBackend.figure_formats = ['svg']

In [ ]:
con = sqlite3.connect('emps.db')

module = pd.read_sql_query("SELECT * FROM module", con)
module_discipline = pd.read_sql_query("SELECT * FROM module_discipline", con)
programme = pd.read_sql_query("SELECT * FROM programme", con)
requisite = pd.read_sql_query("SELECT * FROM requisite", con)
staff = pd.read_sql_query("SELECT * FROM staff", con)
stage_module = pd.read_sql_query("SELECT * FROM stage_module", con)

In [ ]:
RQ_edge_sql = """
WITH filtered_modules AS (
	SELECT DISTINCT
		moduleCode
	FROM
		stage_module
	WHERE
		stage_module.moduleCode IS NOT NULL
		AND moduleTitle NOT LIKE '%elective%'
		AND moduleTitle NOT LIKE '%free choice%'
		AND stage_module.program_id IN(
			SELECT
				id FROM programme
			WHERE
				"year" BETWEEN 2021 AND 2024
				AND title LIKE '%mathematics%'
				AND programme. "type" = "Undergraduate")
)
SELECT
	*
FROM
	"requisite"
WHERE
	requisite. "module" IN filtered_modules
	AND requisite.requires IN filtered_modules;
"""
RQ_edge_df = pd.read_sql_query(RQ_edge_sql, con)
RQ_node_sql = """
SELECT DISTINCT
	MIN(sm.stage) AS stage,
	sm.moduleCode,
	sm.moduleTitle,
	sm.moduleCredits,
	m.credit,
	MIN(sm.isCompulsory) AS IsCompulsory,
	MAX(sm.isNonCondonable) AS IsNonCondonable,
	m.studentsCount,
	m.year
FROM
	stage_module AS sm
	LEFT JOIN "module" AS m ON sm.moduleCode = m.moduleCode
WHERE
	sm.moduleCode IS NOT NULL
	AND sm.moduleTitle NOT LIKE '%elective%'
	AND sm.moduleTitle NOT LIKE '%free choice%'
	AND sm.moduleCode NOT IN('xxxxxxx', 'BE*M**', 'BE*M***', 'BEE3*', 'BEE3***', 'BEE3****', 'BEE3XXX', 'BEM2***', 'BIOMXXX', 'MTH****', 'MTH2XXX/3XXX', 'MTH3***', 'PHYMXXX', 'GEOMXXX')
	AND sm.program_id IN(
		SELECT
			id FROM programme
		WHERE
			"year" BETWEEN 2021 AND 2024
			AND title LIKE '%mathematics%'
			AND programme. "type" = "Undergraduate")
GROUP BY
	sm.moduleCode;
"""
RQ_node_df = pd.read_sql_query(RQ_node_sql, con)

G = nx.DiGraph()

for index, row in RQ_node_df.iterrows():
    node_id = row['moduleCode']
    attributes = {k: v for k, v in row.items() if k != 'moduleCode'}
    G.add_node(node_id, **attributes)

for index, row in RQ_edge_df.iterrows():
    source = row['module']
    target = row['requires']
    attributes = {k: v for k, v in row.items() if k != 'module' and k != 'requires'}
    G.add_edge(source, target, **attributes)
    G.edges[source, target]['label'] = 'requires'

VIS_CENTER = {'avoid_overlap':1.0, 'central_gravity':0.95}
# A = nx.nx_agraph.to_agraph(G)
V = gv.vis(G, node_size_data_source="studentsCount", use_node_size_normalization=True, **VIS_CENTER)
V.export_html("./G.html", True)
V
# A.layout("fdp")
# # A
# # layout = nx.kamada_kawai_layout(G)
# layout = nx.spring_layout(G, iterations=100000)
# layout = nx.spring_layout(G)
# nx.draw(G, pos=layout)
# with open('g.cyjs', 'w') as f:
#     import json
#     json.dump(nx.cytoscape_data(G), f)

# gt.sfdp_layout()


In [ ]:
def overlap_coefficient(X, Y):
    intx = nx.intersection(X, Y)
    return len(intx.nodes) / min(len(X.nodes), len(Y.nodes))

In [ ]:
def jaccard_index(X, Y):
    intersection = nx.intersection(X, Y)
    union = nx.compose(X, Y)
    return len(intersection) / len(union)

In [ ]:
def node_symetric_difference(X, Y):
    intx = nx.intersection(X, Y)
    union = nx.compose(X, Y)
    return len(union.nodes - intx.nodes)

In [ ]:
def compare_laplacian(X, Y):
    X_lap = nx.laplacian_spectrum(X.to_undirected())
    Y_lap = nx.laplacian_spectrum(Y.to_undirected())

    penalty = abs(len(X_lap) - len(Y_lap))
    longest = max(len(X_lap), len(Y_lap))

    X_lap = np.pad(X_lap, (0, longest - len(X_lap)))
    Y_lap = np.pad(Y_lap, (0, longest - len(Y_lap)))

    manhattan_distance = np.sum(np.abs(X_lap - Y_lap))

    return manhattan_distance + penalty

In [ ]:
def compare_adjacency(X, Y):
    X_adj = nx.adjacency_spectrum(X.to_undirected())
    Y_adj = nx.adjacency_spectrum(Y.to_undirected())

    penalty = abs(len(X_adj) - len(Y_adj))
    longest = max(len(X_adj), len(Y_adj))

    X_adj = np.pad(X_adj, (0, longest - len(X_adj)))
    Y_adj = np.pad(Y_adj, (0, longest - len(Y_adj)))

    manhattan_distance = np.sum(np.abs(X_adj - Y_adj))

    return manhattan_distance + penalty

Business Modules
BEE1038
BEM1006
BEM2007
BEM2036
BEM2031
BEM3060
BEM3052

In [ ]:
J_y1 = ["ECM1400", "ECM1410", "ECM1413", "ECM1414", "MTH1001", "MTH1002"]
J_y2 = ["ECM2414", "ECM2418", "ECM2419", "ECM2423", "ECM2433", "ECM2434", "MTH2008", "MTH2011"]
J_y3 = ["ECM3401", "ECM3408", "ECM3420", "ECM3423", "MTH3019", "MTH3022"]
J = G.subgraph(J_y1 + J_y2 + J_y3)

gv.vis(J, **VIS_CENTER)

In [ ]:
L_y1 = ["MTH1001", "MTH1002", "MTH1003", "MTH1004"]
L_y2 = ["MTH2003", "MTH2004", "MTH2008", "MTH2009", "MTH2010", "MTH2011"] # NOTE: and sociology modules
L_y3 = ["MTH3004", "MTH3022", "MTH3019", "MTH3042", "MTH3035"] # NOTE: and sociology modules

L = G.subgraph(L_y1 + L_y2 + L_y3)
gv.vis(L, **VIS_CENTER)

In [ ]:
M_y1 = ["ECM1400", "ECM1410", "ECM1413", "ECM1414", "MTH1001", "MTH1002"]
M_y2 = ["ECM2414", "ECM2418", "MTH2010", "ECM2427" , "ECM2433", "ECM2434", "MTH2008", "MTH2011"]
M_y3 = ["ECM3420", "ECM3428", "MTH3004", "MTH3019", "ECM3408", "ECM3401"]

M = G.subgraph(M_y1 + M_y2 + M_y3)

In [ ]:
BJ_y1 = ["MTH1001", "MTH1002", "MTH1003", "MTH1004"]
BJ_y2 = ["MTH2003", "MTH2004", "MTH2005", "MTH2006", "MTH2008", "MTH2009"]
BJ_y3 = ["BEM2031", "BEM2036", "MTH3024", "MTH3028", "MTH3030", "MTH3035", "MTH3042", "MTH3045"]

BJ = G.subgraph(BJ_y1 + BJ_y2 + BJ_y3)

In [ ]:
JLT_y1 = ["MTH1001", "MTH1002", "MTH1003", "MTH1004"]
JLT_y2 = ["MTH2003", "MTH2006", "MTH2008", "MTH2010", "MTH2011", "BEE1038", "BEM2036"]
JLT_y3 = ["BEM2031", "MTH3019", "MTH3024", "MTH3028", "MTH3045"]

JLT = G.subgraph(JLT_y1 + JLT_y2 + JLT_y3)

In [ ]:
CC_y1 = ["MTH1001", "MTH1002", "MTH1003", "MTH1004"]
CC_y2 = ["MTH2003", "MTH2006", "MTH2008", "MTH2010", "MTH2011", "BEE1038", "MTH2009"]
CC_y3 = ["MTH3028", "BEM2031", "MTH3004", "MTH3024", "MTH3038", "MTH3026", "MTH3022", "MTHM036"]

CC = G.subgraph(CC_y1 + CC_y2 + CC_y3)

In [ ]:
EP_y1 = ["MTH1001", "MTH1002", "MTH1003", "MTH1004"]
EP_y2 = ["MTH2003", "MTH2004", "MTH2005", "MTH2008", "MTH2009", "MTH2010"]
EP_y3 = ["MTH3019", "MTH3022", "MTH3035", "MTH3042", "MTH3007", "MTH3011"] # + German Beginners

EP = G.subgraph(EP_y1 + EP_y2 + EP_y3)

In [ ]:
ZL_y1 = ["MTH1001", "MTH1002", "MTH1003", "MTH1004"]
ZL_y2 = ["MTH2003", "MTH2004", "MTH2005", "MTH2006", "MTH2008", "MTH2009"]
ZL_y3 = ["MTH3035", "MTH3006", "MTH3007", "MTH3042", "MTH3022", "BEM3060", "BEM3052"]

ZL = G.subgraph(ZL_y1 + ZL_y2 + ZL_y3)

In [ ]:
IW_y1 = ["MTH1001", "MTH1002", "MTH1003", "MTH1004"]
IW_y2 = ["MTH2003", "MTH2004", "MTH2005", "MTH2008", "MTH2009"]
IW_y3 = ["MTH3035", "MTH3006", "MTH3007", "MTH3042", "MTH3022", "MTH3011", "BEM3060", "BEM3052"]

IW = G.subgraph(IW_y1 + IW_y2 + IW_y3)

In [ ]:
K_y1 = ["MTH1001", "MTH1002", "MTH1003", "MTH1004"]
K_y2 = ["MTH2003", "MTH2006", "MTH2008", "MTH2009", "MTH2010", "MTH2011", "BEM1006"]
K_y3 = ["MTH3042", "MTH3022", "MTH3024", "MTH3035", "BEM2007", "BEM3060", "MTH3040", "MTH3028"]

K = G.subgraph(K_y1 + K_y2 + K_y3)

In [ ]:
intx = nx.intersection(J, L)
C = nx.compose_all([J, L, M])
for node in C.nodes:
    if node in intx:
        C.nodes[node]['color'] = "red"
    elif node in J:
        C.nodes[node]['color'] = "green"
    elif node in L:
        C.nodes[node]['color'] = "blue"
    else:
        C.nodes[node]['color'] = "black"


gv.vis(C, **VIS_CENTER)

In [ ]:
people = [J, L, M, BJ, JLT, CC, EP, ZL, IW, K]
ALL = nx.compose_all(people).copy()

for node in ALL.nodes:
    score = 0
    for P in people:
        if node in P:
            score += 1
    ALL.nodes[node]["count"] = score

gv.vis(ALL, use_node_size_normalization=True, node_size_data_source="count", **VIS_CENTER)

In [ ]:
def compare_graph_similarity(graphs, functions):
    pairs = list(combinations(range(len(graphs)), 2))
    outputs = {}
    for func in functions:
        results = np.zeros((len(graphs), len(graphs)))
        for (X, Y) in pairs:
            results[X, Y] = results[Y, X] = func(graphs[X], graphs[Y])
        outputs[func.__name__] = results
    return outputs

In [ ]:
people = [J,L, M, JLT, EP, CC, K, ZL, IW, BJ]
funcs = [
    overlap_coefficient,
    jaccard_index,
    node_symetric_difference,
    compare_laplacian,
    compare_adjacency,
]
similarities = compare_graph_similarity(people, funcs)

for n, m in similarities.items():
    with np.printoptions(precision=3, suppress=True):
        print(f"{n} matrix:")
        # print(m)
        print(np.array2string(m, separator=' & ', max_line_width=np.inf, formatter={'int': lambda x: f"{x:d}"}))


distance_matrix = similarities["node_symetric_difference"]

people_names = ["J", "M", "L", "JT", "E", "C", "K", "Z", "I", "B"]
name_mapping = {index: value for index, value in enumerate(people_names)}

H = nx.from_numpy_array(distance_matrix)
nx.relabel_nodes(H, name_mapping, False)

D = H.copy()
for u, v, data in D.edges(data=True):
    if "weight" in data:
        data["weight"] = round(data["weight"], 2)
gv.vis(D, edge_label_data_source="weight", show_edge_label=True)

# [J, M, L, JLT, EP, CC, K, ZL, IW, BJ]

In [ ]:
clustering = DBSCAN(eps=8, min_samples=2, metric='precomputed')
# ['J', 'M', 'L', 'BJ', 'JLT', 'CC', 'K'  'ZL', 'EP','IW']
# [ CS, CS,   ?,  Ap/St, St,   P/St, P/St   Ap,  P/Ap,  P,  ]
clustering.fit(distance_matrix)
clustering.labels_

In [ ]:
# ['J', 'M', 'L', 'BJ', 'JLT', 'CC', 'K'  'ZL', 'EP','IW']
# [ CS, CS,   ?,  Ap/St, St,   P/St, P/St   Ap,  P/Ap,  P,  ]

km = KMeans(n_clusters=3).fit(distance_matrix)
km.labels_

# Knowledge Representation Via Directed Graphs

## Network Representations of various Mathematics Degrees
- Graphs with degree dependencies
- Graphs with degree dependencies + MSC

Consider a directed graph $G$, whose vertices represent modules in the College of Engineering, Mathematics and Physical Sciences (EMPS) at Exeter University. In this graph, the directed edges represent a modules prerequisite. For example if a edge has its tail at "MTH1001" and its head at "MTH2010" it encodes the information that a student must take or have taken Mathematical Structures in order to take Groups, Rings and Fields. These edges will have the value "requires". Later we will add different types of edges to the graph, but we shall stick to just module requirements for now.

The EMPS modules which a mathematics student at Exeter University takes can therefore be represented as a subgraph of $G$, since this graph contains all modules, and module requirements are the same for every student. As part of our investigation we analysed 10 students Mathematics degrees, each taking a variety of different mathematics and non-mathematics modules. Each student was also asked to assign to each module a variety of Mathematical Subject Classification (MSC) codes [CITE]. MSC codes are used to categorise mathematical fields and can be used to represent the type of mathematical content a module covers. Each individual gave their own MSC codes for each module, as different students perception of each module may be different. The MSC module codes have been omitted in the following example for readability.

## Methods for Comparing Similarity
- The Overlap Coefficient, also known as the Szymkiewicz-Simpson coefficient
- Graph Edit Distance (Without MSC)

## Practical Applications
- Guide for new students picking modules
- Improved graduate job searching, e.g. matching similarity of grad cracker or linkedin job posts and degree graph (using LLMs)

## LLM Categorisation
- Problems with LLMs: ChatGPT hallucinates of MSC codes e.g.
68N30 - Mathematical aspects of software engineering in general
68N31 - Predesign
68N32 - Design
- Problems with LLMs: Context length, needs to be able to provide enough tokens (words) to give the whole MSC and also EMPS module description, context length is $O(n^2)$
- Around 40k tokens for MSC code and title
- Around 4K tokens for Module Descriptors
- Solution use Claude, arguably best current model, largest context window and input token limit (200K). Do prompt engineering to avoid hallucinations

```
This is the MSC subject classifications is CSV format:
<MSC>
    {msc_text}
</MSC>

This is a universities module description
<module>
    {module_description}
</module>

Using the MSC subject classifications, classify this module. Only answer if you know the answer with certainty. Justify your answers. List the classification or classifications inside <classification> XML tags.
```


## Machine Learning Methods
- Use DBSCAN to cluster types of degrees based on distances between individuals